## Import Libraries

In [ ]:
# Import necessary libraries for data handling, optimization, and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import time
import pickle
from itertools import permutations
from gurobipy import *
from gurobipy import GRB

In [ ]:
# Set your working directory
my_folder = "C:/Users/hyunwoolee/OneDrive - Virginia Tech/Hyunwoo Research"

## General Functions

In [ ]:
# Converts a set of selected lakes into selfish and global utility values.
# - 'selected_lakes' is a list of lake indices selected by the players (counties).
# - Returns: 
#     1. A dictionary of selfish utilities per county.
#     2. A scalar value representing the global utility (social welfare).

def from_x_to_obj(selected_lakes):
    
    # fix x-variables based on selected_lakes
    x = {i: 1 if i in selected_lakes else 0 for i in I}
    y = {arc: min(1,x[arc[0]]+x[arc[1]]) for arc in arcs}
    
    # Compute selfish objectives for each county
    selfish_obj = {county: sum(n[arc] * y[arc] *t [arc] for arc in arcs if arc[1] in I_c[county]) for county in counties}

    # Compute global objective 
    global_obj = sum(n[arc]*y[arc]*t[arc] for arc in arcs)   

    return selfish_obj, global_obj

In [ ]:
def printSolution(model, x, selected_lakes):
    
    # Check if the model found a feasible solution
    if model.SolCount > 0:
        print('\nObjective value: %g' % model.ObjVal)

        # Append selected lakes (x[i] ≈ 1) to the result list
        for i in I:
            if x[i].X > 0.1:
                # print('Lake %s: %g' % (i, x[i].X))
                selected_lakes.append(i)
    else:
        pass  # No feasible solution found


## Cycle_diagnostic

In [ ]:
def Cycle_Diagnostic(selected_lakes):
    # Initialize flag for cycle detection
    Cycle_found = False
    S = []

    # Check for duplicate lake entries (indicates a cycle)
    for i in range(len(selected_lakes)):
        if selected_lakes[i] in S:
            print('======================================================================================================')
            print('\nCycle detected =====================================================================================\n')
            Cycle_found = True
            return Cycle_found
        S.append(selected_lakes[i])
    
    return Cycle_found

## Social Welfare Model

In [ ]:
### This function builds and solves a Gurobi model that maximizes social welfare

def SW_model(time_limit, usage):
    start_time = time.time()

    # Initialize the Gurobi model
    model = Model("SW model")
    
    # Decision variables
    x = model.addVars(I, vtype=GRB.BINARY, name="x")       # Lake selection variables
    y = model.addVars(arcs, vtype=GRB.BINARY, name="y")    # Arc activation variables

    # Budget constraint: each county's total selected lakes must not exceed its budget
    model.addConstrs((quicksum(x[i] for i in I_c[county]) <= county_budget[county] for county in counties), name="Budget")

    # Arc activation constraint: arc can be active only if at least one endpoint lake is selected
    model.addConstrs((y[arc] <= x[arc[0]] + x[arc[1]] for arc in arcs), name="Link")

    # Objective: maximize total social welfare (weighted arc usage)
    model.setObjective(quicksum(t[arc] * n[arc] * y[arc] for arc in arcs),GRB.MAXIMIZE)

    # Set time limit based on usage type
    if usage == 'OSW':
        model.setParam('TimeLimit', time_limit)
    elif usage == 'OSW_warm':
        model.setParam('TimeLimit', time_limit / 6)

    # Solver parameters
    model.Params.LogToConsole = 0
    model.Params.Threads = user_Threads

    # Solve the model
    model.optimize()
    runtime = model.Runtime

    # Extract selected lakes from solution
    selected_lakes = []
    printSolution(model, x, selected_lakes)

    end_time = time.time()
    elapsed_time = end_time - start_time

    return model, elapsed_time, selected_lakes

## Best-response problem

In [ ]:
### This function defines the best-response problem for a single player (county)

def selfish(county):
    # Initialize a Gurobi model for the county's selfish optimization
    model = Model("Selfish")
    model.Params.LogToConsole = 0
    model.Params.MIPGap = 1e-6
    model.Params.Threads = user_Threads

    # Decision variables: x[i] for all lakes (owned and complementary)
    x = model.addVars(I_c[county] + I_c_complement[county], vtype=GRB.BINARY, name="x")

    # Arc variables: only include arcs relevant to the current county
    relevant_arcs = [arc for arc in arcs if arc[1] in I_c[county]]
    y = model.addVars(relevant_arcs, vtype=GRB.BINARY, name="y")

    # Budget constraint: selected lakes cannot exceed the county's budget
    model.addConstr(quicksum(x[i] for i in I_c[county]) <= county_budget[county])

    # Arc activation constraint: y[arc] can be 1 only if at least one endpoint lake is selected
    model.addConstrs((y[arc] <= x[arc[0]] + x[arc[1]] for arc in relevant_arcs), name="Link")

    # Objective: maximize the county's individual utility
    model.setObjective(quicksum(t[arc] * n[arc] * y[arc] for arc in relevant_arcs), GRB.MAXIMIZE)
    model.update()

    return model

## CR-BRD algorithm

In [ ]:
def BRD(x_current, selfish_model, max_init, max_iteration):
    """
    Executes the Clockwork-Random Best-Response Dynamics (CR-BRD) algorithm for the EBMC game.

    Starting from an initial strategy profile, this function iteratively updates each county's strategy
    using best-response optimization. If cycles are detected, randomization is introduced into the
    playing sequence to escape local loops. The algorithm searches for a Pure Nash Equilibrium (PNE).

    Args:
        x_current (list): Initial strategy profile; a list of selected lakes.
        selfish_model (dict or None): Dictionary of pre-built Gurobi models for each county's
                                      best-response problem. If None, models will be created during execution.
        max_init (int): Maximum number of randomized initializations to attempt.
        max_iteration (int): Maximum number of BRD iterations per initialization.

    Returns:
        tuple:
            selected_lakes_final (list): Flattened list of selected lakes in the final strategy profile.
            BR_PNE_found (bool): True if a Pure Nash Equilibrium was identified.
            Cycle_found (bool): True if a cycle was detected during any iteration.
            total_iterations (int): Total number of best-response updates performed.
            solution (dict): Final objective value for each county in the equilibrium profile.         
    """
    
    global BR_PNE_found, I_c

    # Initialize selected_lakes and solution dictionary
    selected_lakes = {}
    selected_lakes[0] = {}
    for county in counties:
        selected_lakes[0][county] = [i for i in x_current if i in x_current and lake_county[i]==county]
    solution = {}
    solution[0] = from_x_to_obj(x_current)[0]

    # Create a container for selfish models if not already provided
    if selfish_model is None:
        selfish_model = {}
    
    # Initial settings
    playing_sequence = counties
    randomized, Cycle_found = False, False
    total_iterations = 0

    # Try up to max_init different initial strategy profiles
    for num_init in range(1,max_init+1):
        if num_init > 1:
            # Generate a new random strategy profile
            x_current = []
            for county, budget in county_budget.items():
                # Create a list of elements for the current group
                elements = [f"{county}{i}" for i in range(1, len(I_c[county]))]
                x_current.extend(random.sample(elements, budget))

            # Reset initial selections and utilities            
            selected_lakes = {}
            selected_lakes[0] = {}
            for county in counties:
                selected_lakes[0][county] = [i for i in x_current if i in x_current and lake_county[i]==county]
            solution = {}
            solution[0] = from_x_to_obj(x_current)[0]

        # Run best-response updates up to max_iteration times
        for num in range(1,max_iteration+1):
            total_iterations += 1

            selected_lakes[num] = {}
            solution[num] = {}
            for county in counties:
                solution[num][county] = 0
                selected_lakes[num][county] = selected_lakes[num-1][county]

            for county in playing_sequence:
                # Reuse or initialize best-response model for this county
                if county not in selfish_model:
                    selfish_model[county] = selfish(county)
                
                # Fix other counties' strategies (complement lakes)
                selected_lakes_previous = [lake for key, val in selected_lakes[num].items() for lake in val]
                for i in I_c_complement[county]:
                    var = selfish_model[county].getVarByName(f"x[{i}]")
                    if i in selected_lakes_previous:                    
                        var.lb, var.ub = 1, 1
                    else:
                        var.lb, var.ub = 0, 0
                        
                # Warm-starting with the current strategy profile
                for i in I_c[county]:
                    selfish_model[county].getVarByName(f"x[{i}]").start = 1 if i in selected_lakes_previous else 0
                
                selfish_model[county].update()
                selfish_model[county].optimize()

                # Store best-response outcome
                solution[num][county] = selfish_model[county].ObjVal
                selected_lakes[num][county] = [i for i in I_c[county] if selfish_model[county].getVarByName(f"x[{i}]").X > 0.1]

            # Check for convergence to a Pure Nash Equilibrium
            if selected_lakes[len(selected_lakes)-1] == selected_lakes[len(selected_lakes)-2]:
                # print(' PNE found at %d iteration ================================================================== '%(num))
                BR_PNE_found = True
                selected_lakes_final = [lake for key,val in selected_lakes[num].items() for lake in val ]

                return selected_lakes_final, BR_PNE_found, Cycle_found, total_iterations, solution[num]  
        
            # Enable random play order if a cycle is detected
            if randomized:
                counties_tmp = counties.copy()
                random.shuffle(counties_tmp)
                playing_sequence = counties_tmp
            else:
                Cycle_found = Cycle_Diagnostic(selected_lakes) 
                if Cycle_found:                    
                    randomized = True
                    counties_tmp = counties.copy()
                    random.shuffle(counties_tmp)
                    playing_sequence = counties_tmp

    # No equilibrium found after all initializations and iterations        
    return [], False, True, total_iterations, {county:0 for county in counties}

## Separation Oracle

In [ ]:
### This callback function integrates a separation oracle to enforce equilibrium inequalities

def Separation(model, where):
    global x, y, EI_cuts, selfish_model, first_PNE_obj, first_time, start_time, PNE_sets

    violated = False

    # Trigger only when a new feasible integer solution is found
    if where == GRB.Callback.MIPSOL:
        # Extract current incumbent solution
        x_current = [key for key,item in x.items() if model.cbGetSolution(x[key]) > 0.3]
        curr_selfish_obj, curr_global_obj = from_x_to_obj(x_current)    
        
        for county in counties:
            # Reuse or create a selfish model for this county
            if county not in selfish_model:
                selfish_model[county] = selfish(county)

            # Fix other counties’ decisions in the model
            for i in I_c_complement[county]:
                var = selfish_model[county].getVarByName(f"x[{i}]")
                if i in x_current:                    
                    var.ub, var.lb = 1, 1
                else:                    
                    var.ub, var.lb = 0, 0
            
            # Warm-starting with the current strategy profile
            for i in I_c[county]:
                selfish_model[county].getVarByName(f"x[{i}]").start = 1 if i in x_current else 0

            # Solve the county’s best-response problem
            selfish_model[county].optimize()

            # If a better response exists, add a lazy cut to eliminate the deviation
            if selfish_model[county].ObjVal > curr_selfish_obj[county]:            
                best_response_lakes = [i for i in I_c[county] if selfish_model[county].getVarByName(f"x[{i}]").X > 0.1]                
                # print(f'Best utility: { best_response_objval }')
                # print(f'Current utility: {sum(t[arc]*n[arc]*model.cbGetSolution(u[arc]) for arc in arcs if arc[1] in I_c[county])}')                
                # print('Best utility is higher than current utility')
                # print("\n"+"="*35 + "  Call Lazy cuts  " + "="*35)
            
                # Add equilibrium inequality (lazy cut)
                model.cbLazy(
                    sum(t[arc]*n[arc]*1 for arc in arcs_c[county] if (arc[0] in best_response_lakes) or (arc[1] in best_response_lakes))  
                    + sum(t[arc]*n[arc]*1 for arc in arcs_plus_c[county] if (arc[1] in best_response_lakes))
                    +quicksum(t[arc]*n[arc]*x[arc[0]] for arc in arcs_plus_c[county] if (arc[1] not in best_response_lakes)) 
                    <= quicksum(t[arc]*n[arc]*y[arc] for arc in arcs if arc[1] in I_c[county])
                )                
                EI_cuts.append(1)
                violated = True
                
            # print("\n"+"="*35 + "  Continue Main program  " + "="*35)
            # print("    Nodes    |    Current Node    |     Objective Bounds      |     Work") 
            # print(" Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time")

        # Record the first PNE (Pure Nash Equilibrium) and its discovery time
        if violated == False and first_time == None:
            first_PNE_obj = curr_global_obj 
            first_time = time.time() - start_time
            
        # Store the new PNE if it's unique
        if (violated == False) and (x_current not in PNE_sets):
            PNE_sets.append(x_current)


## CR-BRD Incorporated Separation Oracle

In [ ]:
### This callback function integrates both a separation oracle and the CR-BRD algorithm

def BRD_Separation(model, where):
    global x, y, EI_cuts, PNE_cuts, selfish_model, first_PNE_obj, first_time, start_time, best_obj, BR_sets, PNE_sets

    violated = False

    # Trigger only when a new feasible integer solution is found
    if where == GRB.Callback.MIPSOL:
        # Extract the current solution (selected lakes)
        x_current = [key for key,item in x.items() if model.cbGetSolution(x[key]) > 0.3]
        curr_selfish_obj, curr_global_obj = from_x_to_obj(x_current)    
        
        for county in counties:
            # Reuse or initialize selfish model for the county
            if county not in selfish_model:
                selfish_model[county] = selfish(county)

            # Fix other counties’ decisions in the model
            for i in I_c_complement[county]:
                var = selfish_model[county].getVarByName(f"x[{i}]")
                if i in x_current:                    
                    var.ub, var.lb = 1, 1
                else:                    
                    var.ub, var.lb = 0, 0
            
            # Warm-starting with the current strategy profile
            for i in I_c[county]:
                selfish_model[county].getVarByName(f"x[{i}]").start = 1 if i in x_current else 0

            # Solve the county's best-response problem
            selfish_model[county].optimize()
            
            # If a better response exists, add a lazy cut to eliminate the deviation
            if selfish_model[county].ObjVal > curr_selfish_obj[county]:
                best_response_lakes = [i for i in I_c[county] if selfish_model[county].getVarByName(f"x[{i}]").X > 0.1]                
                # print(f'Best utility: { best_response_objval }')
                # print(f'Current utility: {sum(t[arc]*n[arc]*model.cbGetSolution(u[arc]) for arc in arcs if arc[1] in I_c[county])}')                
                # print('Best utility is higher than current utility')
                # print("\n"+"="*35 + "  Call Lazy cuts  " + "="*35)
            
                # Add equilibrium inequality (lazy cut)
                model.cbLazy(
                    sum(t[arc]*n[arc]*1 for arc in arcs_c[county] if (arc[0] in best_response_lakes) or (arc[1] in best_response_lakes))  
                    + sum(t[arc]*n[arc]*1 for arc in arcs_plus_c[county] if (arc[1] in best_response_lakes))
                    +quicksum(t[arc]*n[arc]*x[arc[0]] for arc in arcs_plus_c[county] if (arc[1] not in best_response_lakes)) 
                    <= quicksum(t[arc]*n[arc]*y[arc] for arc in arcs if arc[1] in I_c[county])
                )                
                EI_cuts.append(1)
                violated = True
                
        # Record the first PNE (Pure Nash Equilibrium) and its discovery time
        if violated == False and first_time == None:
            first_PNE_obj = curr_global_obj 
            first_time = time.time() - start_time
            
        # print("\n"+"="*35 + "  Continue Main program  " + "="*35)
        # print("    Nodes    |    Current Node    |     Objective Bounds      |     Work") 
        # print(" Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time")

        # Apply CR-BRD if the current solution is a non-PNE
        if violated == False:
            BRD_lakes, BR_PNE_found, current_sol = x_current, True, curr_selfish_obj
        else:        
            BRD_lakes, BR_PNE_found, Cycle_found, total_iterations, current_sol  = BRD(x_current, selfish_model, max_init = 5, max_iteration = 20)

        # If a new PNE is found through separation or CR-BRD
        if (BR_PNE_found == True) and (BRD_lakes not in PNE_sets):
            curr_global_obj = sum(current_sol[county] for county in counties)
            for county in counties:
                best_response_lakes = [i for i in I_c[county] if i in BRD_lakes]
                
                # If this best response for a county hasn't been found before, add a PNE inequality
                if best_response_lakes not in BR_sets[county]:
                    # Add PNE inequality (lazy cut)
                    model.cbLazy(
                        sum(t[arc]*n[arc]*1 for arc in arcs_c[county] if (arc[0] in best_response_lakes) or (arc[1] in best_response_lakes))  
                        + sum(t[arc]*n[arc]*1 for arc in arcs_plus_c[county] if (arc[1] in best_response_lakes))
                        +quicksum(t[arc]*n[arc]*x[arc[0]] for arc in arcs_plus_c[county] if (arc[1] not in best_response_lakes)) 
                        <= quicksum(t[arc]*n[arc]*y[arc] for arc in arcs if arc[1] in I_c[county])
                    )                
                    BR_sets[county].append(best_response_lakes)
                    PNE_cuts.append(1)

            # Update primal solution if this PNE improves the global objective
            if curr_global_obj > best_obj:
                best_obj = curr_global_obj
                for i in I:
                    if i in BRD_lakes:
                        model.cbSetSolution(x[i], 1)
                    else:
                        model.cbSetSolution(x[i], 0)
                        
            # Track this new PNE
            PNE_sets.append(BRD_lakes)
            
            # Store the new PNE if it's unique
            if first_PNE_obj == None:
                first_PNE_obj = curr_global_obj 
                first_time = time.time() - start_time

## ZR and BZR algorithms

In [ ]:
def ZR(time_limit, callback_type, dataset):
    """
    Zero-Regret (ZR) algorithm solver for the EBMC Game using Gurobi.

    Parameters:
        time_limit (float): Total time allowed for optimization (in seconds).
        callback_type (str): 
            - 'Separation' for the original ZR algorithm with lazy constraints.
            - 'BRD_Separation' for the best-response dynamics–incorporated ZR (BZR algorithm).
        dataset (str): Indicates dataset source, e.g., 'Random' or 'Minnesota'.
        
    Returns:
        model (gurobipy.Model): The optimized Gurobi model.
        elapsed_time (float): Total wall-clock time used.
        selected_lakes (list): Lakes selected in the final solution.
        first_PNE_obj (float): Objective value of the first identified PNE (if found).
        first_time (float): Time when the first PNE was identified.
        cuts_summary (list): [# of equilibrium cuts, # of PNE cuts].
        PNE_sets (list): List of all unique PNEs identified.
    """
    
    global x, y, EI_cuts, PNE_cuts, selfish_model, first_PNE_obj, first_time, start_time, best_obj, BR_sets, PNE_sets

    start_time = time.time()

    # Initialize storage for cuts, models, and tracking info
    EI_cuts, PNE_cuts = [], []
    first_PNE_obj, first_time, best_obj = None, None, -10000
    selfish_model = {}
    BR_sets, PNE_sets = {county:[] for county in counties}, []
    
    # Build Gurobi model
    model = Model(f"ZR_model")

    # Decision variables
    x = model.addVars(I, vtype=GRB.BINARY, name="x")         # Lake selection
    y = model.addVars(arcs, vtype=GRB.BINARY, name="y")      # Arc activation

    # Constraints
    model.addConstrs(y[arc] <= (x[arc[0]] + x[arc[1]]) for arc in arcs)  # Arc linkage constraint
    model.addConstrs(quicksum(x[i] for i in I_c[county]) <= county_budget[county] for county in county_budget)  # County-level budget constraint
                
    # Objective: maximize total social welfare
    model.setObjective(quicksum(t[arc] * n[arc] * y[arc] for arc in arcs), GRB.MAXIMIZE)

    # Gurobi solver parameters
    model.Params.LazyConstraints = 1
    model.Params.Threads = user_Threads
    model.Params.LogToConsole = 0
    
    # === Set Log File === #
    if callback_type == 'Separation':
        if dataset == 'Random':
            model.setParam("LogFile", f"Log_ZR_{county_size}_{num_lakes_per_county}_{budget_ratio}_{type_dataset}")
        elif dataset == 'Minnesota':
            model.setParam("LogFile", "Log_ZR_minnesota")
    elif callback_type == 'BRD_Separation':
        if dataset == 'Random':
            model.setParam("LogFile", f"Log_BZR_{county_size}_{num_lakes_per_county}_{budget_ratio}_{type_dataset}")
        elif dataset == 'Minnesota':
            model.setParam("LogFile", "Log_BZR_minnesota")       

    # Update time limit to account for setup time
    remaining_time = time_limit - (time.time() - start_time)
    model.setParam('TimeLimit', remaining_time)

    # Launch solver with appropriate callback
    if callback_type == 'Separation':
        model.optimize(Separation)
    elif callback_type == 'BRD_Separation':
        model.optimize(BRD_Separation)
        
    runtime=model.Runtime
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    # Extract final lake selections
    selected_lakes = []
    printSolution(model,x,selected_lakes)
    
    # Return results
    return model, elapsed_time, selected_lakes, first_PNE_obj, first_time, [len(EI_cuts), len(PNE_cuts)], PNE_sets

## Experiment Loop: Run All Settings for Random Dataset

In [ ]:
county_size_set = [2,3,5,8,10,15,20,25,30]
num_lakes_per_county_set = [50]
budget_ratio_set = [0.3,0.5,0.8]
datasets = ['single','multi']

standard_time_limit = 1800
user_Threads = 16
global type_dataset 

In [ ]:
for type_dataset in datasets:
    for county_size in county_size_set:
        for num_lakes_per_county in num_lakes_per_county_set:
            for budget_ratio in budget_ratio_set:
                
                # === Load Dataset === #
                df_edge = pd.read_csv(my_folder+f"/BZR_EBMC/EBMC_generated/{type_dataset}_dataset/{county_size}_{num_lakes_per_county}_{budget_ratio}.csv", index_col=0)
                with open(my_folder+f"/BZR_EBMC/EBMC_generated/{type_dataset}_dataset/info_data.pickle",'rb') as f:
                    info_data = pickle.load(f)
        
                # === Unpack Experiment Settings === #
                counties = info_data[(county_size,num_lakes_per_county,budget_ratio)][0] 
                num_lakes_per_county = info_data[(county_size,num_lakes_per_county,budget_ratio)][1]
                infestation_status = info_data[(county_size,num_lakes_per_county,budget_ratio)][2]
                county_budget = info_data[(county_size,num_lakes_per_county,budget_ratio)][3]   
                infested_lakes = [key for key, values in infestation_status.items() if any(val > 0 for val in values.values())]
        
                lakes = np.unique(np.concatenate((df_edge['dow_origin'].unique(),df_edge['dow_destination'].unique()),0))
                lake_county = {lake: lake[:2] for lake in lakes}
        
                # === Compute Lake Weights === #
        
                w={}
                for i in lakes:
                    w[i]=0
                for index,row in df_edge.iterrows():
                    if row['bij'] != 0:
                        w[row['dow_origin']] += row['bij']*row['weight']
                        w[row['dow_destination']] += row['bij']*row['weight']
        
                # === Set Model Parameters === #
        
                I = lakes
                I_c = {county: [i for i in I if i[:2] == county] for county in counties}
                I_c_complement = {county: [i for i in I if i not in I_c[county]] for county in counties}
        
                arcs, n, t = [], {}, {}
        
                for index in df_edge.index:
                    n[df_edge.loc[index,'dow_origin'], df_edge.loc[index,'dow_destination']] = df_edge.loc[index,'weight']
                    t[df_edge.loc[index,'dow_origin'], df_edge.loc[index,'dow_destination']] = df_edge.loc[index,'bij']
        
                arcs = list(n.keys())
        
                # arcs_c means arcs within county c
                # arcs_plus_c means incoming arcs to county c
                # arcs_minus_c means outgoing arcs from county c
        
                arcs_c, arcs_plus_c, arcs_minus_c = {}, {}, {}
        
                for county in counties:
                    arcs_c[county] = [arc for arc in arcs if (arc[0] in I_c[county]) and (arc[1] in I_c[county])]
                    arcs_plus_c[county] = [arc for arc in arcs if (arc[1] in I_c[county]) and (arc[0] in I_c_complement[county])]
                    arcs_minus_c[county] = [arc for arc in arcs if (arc[0] in I_c[county]) and (arc[1] in I_c_complement[county])]


                # === Run OSW Model === #
                print("="*35 + "  SW_model  " + "="*35)
                OSW, elapsed_time_OSW, OSW_selected_lakes = SW_model(standard_time_limit, 'OSW')
                selfish_obj_OSW, global_obj_OSW = from_x_to_obj(OSW_selected_lakes)
                print('global obj: ',global_obj_OSW)
        
        
                # === Run BRD (initial strategy profile: 0) === #
                print("="*35 + "  BRD(0)_model  " + "="*35)
        
                x_current = []
                start_time = time.time()
                first_PNE, PNE_found_BRD_0, Cycle_found_BRD_0, iterations_BRD_0, obj = BRD(x_current, None, max_init=10, max_iteration=20)
                end_time = time.time()
                time_BRD_0 = end_time - start_time
                selfish_obj_BRD_0, global_obj_BRD_0 = from_x_to_obj(first_PNE)
                print('global obj: ',global_obj_BRD_0)
        
        
                # === Run BRD (initial strategy profile: time-limited OSW solution) === #
                print("="*35 + "  BRD(OSW)_model  " + "="*35)
                
                OSWW, time_OSWW, OSWW_selected_lakes = SW_model(standard_time_limit, 'OSW_warm')
                start_time = time.time()
                near_best_PNE, PNE_found_BRD_OSW, Cycle_found_BRD_OSW, iterations_BRD_OSW, obj  = BRD(OSWW_selected_lakes, None, max_init=10, max_iteration=20)
                end_time = time.time()
                time_BRD_OSW = end_time - start_time + time_OSWW
                selfish_obj_BRD_OSW, global_obj_BRD_OSW = from_x_to_obj(near_best_PNE)
                print('global obj: ',global_obj_BRD_OSW)
        
            
                # === Run ZR === #
                print("="*35 + "  ZR_model  " + "="*35)
                model_ZR, time_ZR, best_PNE_ZR, global_obj_first_ZR, first_time_ZR, num_cuts_ZR, PNE_sets_ZR = ZR(standard_time_limit, 'Separation', 'Random')
                selfish_obj_ZR, global_obj_ZR = from_x_to_obj(best_PNE_ZR)
                
                # === Run BZR === #
                print("="*35 + "  BZR  " + "="*35)   
                model_BZR, time_BZR, best_PNE_BZR, global_obj_first_BZR, first_time_BZR, num_cuts_BZR, PNE_sets_BZR = ZR(standard_time_limit, 'BRD_Separation', 'Random')
                selfish_obj_BZR, global_obj_BZR = from_x_to_obj(best_PNE_BZR)

                # Fallback: if the first PNE found by BZR has a higher objective than the final best, use the last PNE identified. 
                # This situation can occur in large-scale models (e.g., ≥ 25 players), where Gurobi's barrier method takes significant time. 
                # As a result, even if CR-BRD identifies a strong solution, the solver might miss recording it due to time limit termination.
                if global_obj_first_BZR!= None and global_obj_first_BZR > global_obj_BZR:
                    best_PNE_BZR = PNE_sets_BZR[-1]  
                selfish_obj_BZR, global_obj_BZR = from_x_to_obj(best_PNE_BZR)


                ##################################  Summarize Results ####################################
                # Create a DataFrame with specified indexes and columns
                columns = ['num_cts','num_lakes','num_infested_lakes','budget_ratio','budget',
                           'ZR_1st','ZR','BZR_1st','BZR','OSW','bound_ZR','bound_BZR','Cuts_ZR', 'Cuts_BZR','PNE_Cuts_BZR',
                           'ZR_1st_T','ZR_T','BZR_1st_T','BZR_T','num_PNEs_ZR','num_PNEs_BZR','Status_ZR','Status_BZR',
                           'BRD(0)','BRD(OSW)','BRD(0)_T','BRD(OSW)_T','it_BRD(0)', 'it_BRD(OSW)', 'Cycle_BRD(0)', 'Cycle_BRD(OSW)']   
                
                # Create the DataFrame
                df_tmp_test = pd.DataFrame(index=['Total'], columns=columns)
        
                df_tmp_test.loc['Total','num_cts'] = len(counties)
                df_tmp_test.loc['Total','num_lakes'] = num_lakes_per_county*len(counties)
                df_tmp_test.loc['Total','num_infested_lakes'] = sum(len([lake for lake in infested_lakes if lake[:2] ==county]) for county in counties)
                df_tmp_test.loc['Total','budget_ratio'] = budget_ratio
                df_tmp_test.loc['Total','budget'] = sum(county_budget[county] for county in counties)
                df_tmp_test.loc['Total','ZR_1st'] = global_obj_first_ZR
                df_tmp_test.loc['Total','ZR'] = global_obj_ZR
                df_tmp_test.loc['Total','BZR_1st'] = global_obj_first_BZR
                df_tmp_test.loc['Total','BZR'] = global_obj_BZR
                df_tmp_test.loc['Total','OSW'] = global_obj_OSW
                df_tmp_test.loc['Total','bound_ZR'] = str(model_ZR.ObjBound)
                df_tmp_test.loc['Total','bound_BZR'] = str(model_BZR.ObjBound)
                df_tmp_test.loc['Total','Cuts_ZR'] = num_cuts_ZR[0]
                df_tmp_test.loc['Total','Cuts_BZR'] = num_cuts_BZR[0]
                df_tmp_test.loc['Total','PNE_Cuts_BZR'] = num_cuts_BZR[1]
                df_tmp_test.loc['Total','ZR_1st_T'] = first_time_ZR
                df_tmp_test.loc['Total','ZR_T'] = time_ZR
                df_tmp_test.loc['Total','BZR_1st_T'] = first_time_BZR
                df_tmp_test.loc['Total','BZR_T'] = time_BZR
                df_tmp_test.loc['Total','num_PNEs_ZR'] = len(PNE_sets_ZR)
                df_tmp_test.loc['Total','num_PNEs_BZR'] = len(PNE_sets_BZR)
                df_tmp_test.loc['Total','Status_ZR'] = str(model_ZR.Status)
                df_tmp_test.loc['Total','Status_BZR'] = str(model_BZR.Status)

                df_tmp_test.loc['Total','BRD(0)'] = global_obj_BRD_0
                df_tmp_test.loc['Total','BRD(OSW)'] = global_obj_BRD_OSW
                df_tmp_test.loc['Total','BRD(0)_T'] = time_BRD_0
                df_tmp_test.loc['Total','BRD(OSW)_T'] = time_BRD_OSW
                df_tmp_test.loc['Total','it_BRD(0)'] = iterations_BRD_0
                df_tmp_test.loc['Total','it_BRD(OSW)'] = iterations_BRD_OSW
                df_tmp_test.loc['Total','Cycle_BRD(0)'] = Cycle_found_BRD_0
                df_tmp_test.loc['Total','Cycle_BRD(OSW)'] = Cycle_found_BRD_OSW
        
                # === Save Results to CSV === #
                results_path = f"{my_folder}/BZR_EBMC/EBMC_results/{type_dataset}_dataset/EBMC_test.csv"
                try:
                    df_test = pd.read_csv(results_path, index_col=0)
                    df_test = pd.concat([df_test, df_tmp_test])
                except FileNotFoundError:
                    df_test = df_tmp_test

                df_test.to_csv(results_path)

## Experiment for Minnesota Dataset

In [ ]:
# ======================= Minnesota Dataset Experiment ======================= #

dataset = 'Minnesota'
standard_time_limit = 3600
user_Threads = 16

# Load data
df_edge = pd.read_csv(my_folder + "/BZR_EBMC/EBMC_minnesota/edgelist1.csv")    
budget_count = pd.read_excel(my_folder + "/BZR_EBMC/EBMC_minnesota/Number of inspection stations.xlsx", header=1)    

# --- Preprocessing for edge data ---
counties = np.unique(np.concatenate((df_edge['county_name.origin'].unique(), df_edge['county_name.destination'].unique()), axis=0))
county_names = counties
lakes = np.unique(np.concatenate((df_edge['dow_origin'].unique(), df_edge['dow_destination'].unique()), axis=0))

lake_county = {}
for lake in lakes:
    if len(df_edge[df_edge['dow_origin'] == lake]) > 0:
        lake_county[lake] = df_edge[df_edge['dow_origin'] == lake]['county_name.origin'].iloc[0]
    else:
        lake_county[lake] = df_edge[df_edge['dow_destination'] == lake]['county_name.destination'].iloc[0]

county_lakes = {county: [] for county in county_names}
for lake in lakes:
    origin_county = df_edge[df_edge['dow_origin'] == lake]['county_name.origin']
    if not origin_county.empty:
        county_lakes[origin_county.iloc[0]].append(lake)
    else:
        dest_county = df_edge[df_edge['dow_destination'] == lake]['county_name.destination'].iloc[0]
        county_lakes[dest_county].append(lake)

# --- Preprocessing for budget data ---
budget_count = budget_count.drop(columns='Unnamed: 0')
budget_count['Number of inspection stations'] = round(budget_count['Number of inspection stations'], 0)
budget_count['County'] = budget_count['County'].str.lower()

county_budget = {}
for county in county_names:
    num = int(budget_count[budget_count['County'] == county]['Number of inspection stations'].values[0])
    county_budget[county] = num

# --- Infested lakes ---
infested_lakes = df_edge[
    (df_edge['zm2019.origin'] == 1) |
    (df_edge['ss2019.origin'] == 1) |
    (df_edge['ew2019.origin'] == 1) |
    (df_edge['sf2019.origin'] == 1)
]['dow_origin'].unique()

num_infested_lakes = {county: 0 for county in counties}
for lake in infested_lakes:
    if lake in lake_county:
        num_infested_lakes[lake_county[lake]] += 1

# --- Weight calculation ---
w = {i: 0 for i in lakes}
for _, row in df_edge.iterrows():
    if row['bij'] != 0:
        w[row['dow_origin']] += row['bij'] * row['weight']
        w[row['dow_destination']] += row['bij'] * row['weight']

# --- Parameter settings for models ---
I = lakes
I_c = {c: [i for i in I if lake_county[i] == c] for c in counties}
I_c_complement = {c: [i for i in I if i not in I_c[c]] for c in counties}

n = {(row['dow_origin'], row['dow_destination']): row['weight'] for _, row in df_edge.iterrows()}
t = {(row['dow_origin'], row['dow_destination']): row['bij'] for _, row in df_edge.iterrows()}
arcs = list(n.keys())

arcs_c = {c: [a for a in arcs if a[0] in I_c[c] and a[1] in I_c[c]] for c in counties}
arcs_plus_c = {c: [a for a in arcs if a[1] in I_c[c] and a[0] in I_c_complement[c]] for c in counties}
arcs_minus_c = {c: [a for a in arcs if a[0] in I_c[c] and a[1] in I_c_complement[c]] for c in counties}


In [ ]:
# === Run OSW Model === #
print("="*35 + "  SW_model  " + "="*35)

start_time = time.time()
OSW, elapsed_time_OSW, OSW_selected_lakes = SW_model(standard_time_limit, 'OSW')
selfish_obj_OSW, global_obj_OSW = from_x_to_obj(OSW_selected_lakes)
print('global obj: ',global_obj_OSW)

# === Run BRD (initial strategy profile: 0) === #
print("="*35 + "  BRD(0)_model  " + "="*35)

x_current = []
start_time = time.time()
first_PNE, PNE_found_BRD_0, Cycle_found_BRD_0, iterations_BRD_0, obj = BRD(x_current, None, max_init=10, max_iteration=20)
end_time = time.time()
time_BRD_0 = end_time - start_time
selfish_obj_BRD_0, global_obj_BRD_0 = from_x_to_obj(first_PNE)
print('global obj: ',global_obj_BRD_0)

# === Run BRD (initial strategy profile: time-limited OSW solution) === #
print("="*35 + "  BRD(OSW)_model  " + "="*35)

OSWW, time_OSWW, OSWW_selected_lakes = SW_model(standard_time_limit, 'OSW_warm')
start_time = time.time()
near_best_PNE, PNE_found_BRD_OSW, Cycle_found_BRD_OSW, iterations_BRD_OSW, obj  = BRD(OSWW_selected_lakes, None, max_init=10, max_iteration=20)
end_time = time.time()
time_BRD_OSW = end_time - start_time + time_OSWW
selfish_obj_BRD_OSW, global_obj_BRD_OSW = from_x_to_obj(near_best_PNE)
print('global obj: ',global_obj_BRD_OSW)

# === Run ZR with Basic Separation Oracle === #
print("="*35 + "  ZR_model  " + "="*35)
model_ZR, time_ZR, best_PNE_ZR, global_obj_first_ZR, first_time_ZR, num_cuts_ZR, PNE_sets_ZR = ZR(standard_time_limit, 'Separation', 'Minnesota')
selfish_obj_ZR, global_obj_ZR = from_x_to_obj(best_PNE_ZR)

# === Run ZR with CR-BRD Enhanced Separation (BZR) === #
print("="*35 + "  BZR  " + "="*35)   
model_BZR, time_BZR, best_PNE_BZR, global_obj_first_BZR, first_time_BZR, num_cuts_BZR, PNE_sets_BZR = ZR(standard_time_limit, 'BRD_Separation', 'Minnesota')
selfish_obj_BZR, global_obj_BZR = from_x_to_obj(best_PNE_BZR)

# Fallback: if the first PNE found by BZR has a higher objective than the final best, use the last PNE identified. 
# This situation can occur in large-scale models (e.g., ≥ 25 players), where Gurobi's barrier method takes significant time. 
# As a result, even if CR-BRD identifies a strong solution, the solver might miss recording it due to time limit termination.
if global_obj_first_BZR!= None and global_obj_first_BZR > global_obj_BZR:
    best_PNE_BZR = PNE_sets_BZR[-1]  
selfish_obj_BZR, global_obj_BZR = from_x_to_obj(best_PNE_BZR)


##################################  Summarize Results ####################################

# Create a DataFrame with specified indexes and columns
columns = ['num_cts','num_lakes','num_infested_lakes','budget',
           'ZR_1st','ZR','BZR_1st','BZR','OSW','bound_ZR','bound_BZR','Cuts_ZR', 'Cuts_BZR','PNE_Cuts_BZR',
           'ZR_1st_T','ZR_T','BZR_1st_T','BZR_T','num_PNEs_ZR','num_PNEs_BZR','Status_ZR','Status_BZR',
           'BRD(0)','BRD(OSW)','BRD(0)_T','BRD(OSW)_T','it_BRD(0)', 'it_BRD(OSW)', 'Cycle_BRD(0)', 'Cycle_BRD(OSW)']   

# Create the DataFrame
df_tmp_test = pd.DataFrame(index=['Total'], columns=columns)

df_tmp_test.loc['Total','num_cts'] = len(counties)
df_tmp_test.loc['Total','num_lakes'] = len(lakes)
df_tmp_test.loc['Total','num_infested_lakes'] = sum(len([lake for lake in infested_lakes if lake[:2] ==county]) for county in counties)
df_tmp_test.loc['Total','budget'] = sum(county_budget[county] for county in counties)
df_tmp_test.loc['Total','ZR_1st'] = global_obj_first_ZR
df_tmp_test.loc['Total','ZR'] = global_obj_ZR
df_tmp_test.loc['Total','BZR_1st'] = global_obj_first_BZR
df_tmp_test.loc['Total','BZR'] = global_obj_BZR
df_tmp_test.loc['Total','OSW'] = global_obj_OSW
df_tmp_test.loc['Total','bound_ZR'] = str(model_ZR.ObjBound)
df_tmp_test.loc['Total','bound_BZR'] = str(model_BZR.ObjBound)
df_tmp_test.loc['Total','Cuts_ZR'] = num_cuts_ZR[0]
df_tmp_test.loc['Total','Cuts_BZR'] = num_cuts_BZR[0]
df_tmp_test.loc['Total','PNE_Cuts_BZR'] = num_cuts_BZR[1]
df_tmp_test.loc['Total','ZR_1st_T'] = first_time_ZR
df_tmp_test.loc['Total','ZR_T'] = time_ZR
df_tmp_test.loc['Total','BZR_1st_T'] = first_time_BZR
df_tmp_test.loc['Total','BZR_T'] = time_BZR
df_tmp_test.loc['Total','num_PNEs_ZR'] = len(PNE_sets_ZR)
df_tmp_test.loc['Total','num_PNEs_BZR'] = len(PNE_sets_BZR)
df_tmp_test.loc['Total','Status_ZR'] = str(model_ZR.Status)
df_tmp_test.loc['Total','Status_BZR'] = str(model_BZR.Status)
df_tmp_test.loc['Total','BRD(0)'] = global_obj_BRD_0
df_tmp_test.loc['Total','BRD(OSW)'] = global_obj_BRD_OSW
df_tmp_test.loc['Total','BRD(0)_T'] = time_BRD_0
df_tmp_test.loc['Total','BRD(OSW)_T'] = time_BRD_OSW
df_tmp_test.loc['Total','it_BRD(0)'] = iterations_BRD_0
df_tmp_test.loc['Total','it_BRD(OSW)'] = iterations_BRD_OSW
df_tmp_test.loc['Total','Cycle_BRD(0)'] = Cycle_found_BRD_0
df_tmp_test.loc['Total','Cycle_BRD(OSW)'] = Cycle_found_BRD_OSW

# === Save Results to CSV === #
results_path = f"{my_folder}/BZR_EBMC/EBMC_results/minnesota/minnesota.csv"
try:
    df_test = pd.read_csv(results_path, index_col=0)
    df_test = pd.concat([df_test, df_tmp_test])
except FileNotFoundError:
    df_test = df_tmp_test

df_test.to_csv(results_path)
